# Local ColabNAS - Exact Configuration

Using the complete flower dataset with your exact specifications.

In [7]:
import tensorflow as tf
import numpy as np
from pathlib import Path
from local_colabnas import LocalColabNAS, test_tflite_model

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU available: {tf.config.list_physical_devices('GPU')}")

TensorFlow version: 2.10.1
GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [8]:
# Download and setup dataset - exact as requested
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos.tar', origin=dataset_url, extract=True)
data_dir = Path(data_dir).with_suffix('')

# Fix dataset structure - point to actual flower_photos folder
actual_data_dir = data_dir / 'flower_photos'
if actual_data_dir.exists():
    data_dir = actual_data_dir

print(f"Dataset directory: {data_dir}")
print(f"Classes found: {[d.name for d in data_dir.iterdir() if d.is_dir()]}")

Dataset directory: C:\Users\Chakkaphan\.keras\datasets\flower_photos
Classes found: ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


In [9]:
# Exact configuration from your request
input_shape = (50,50,3)

#target: STM32L412KBU3
#273 CoreMark, 40 kiB RAM, 128 kiB Flash
peak_RAM_upper_bound = 40960
Flash_upper_bound = 131072
MACC_upper_bound = 2730000 #CoreMark * 1e4

path_to_training_set = data_dir
val_split = 0.3
cache = True
save_path = './trained_models/'

print(f"Configuration:")
print(f"  Input shape: {input_shape}")
print(f"  RAM limit: {peak_RAM_upper_bound} bytes")
print(f"  Flash limit: {Flash_upper_bound} bytes")
print(f"  MACC limit: {MACC_upper_bound}")

Configuration:
  Input shape: (50, 50, 3)
  RAM limit: 40960 bytes
  Flash limit: 131072 bytes
  MACC limit: 2730000


In [10]:
# GPU check (equivalent to !nvidia-smi)
print("GPU Information:")
if tf.config.list_physical_devices('GPU'):
    for gpu in tf.config.list_physical_devices('GPU'):
        print(f"  {gpu}")
else:
    print("  No GPU detected - using CPU")

# Initialize ColabNAS (using LocalColabNAS)
colabNAS = LocalColabNAS(
    max_RAM=peak_RAM_upper_bound, 
    max_Flash=Flash_upper_bound, 
    max_MACC=MACC_upper_bound, 
    path_to_training_set=str(path_to_training_set), 
    val_split=val_split, 
    cache=cache, 
    input_shape=input_shape, 
    save_path=save_path
)

print("\nColabNAS initialized successfully!")


ColabNAS initialized successfully!


In [11]:
tf.config.list_physical_devices('CPU')

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [12]:
# # Search for optimal architecture
# print("Starting search...")
# path_to_tflite_model = colabNAS.search()

# if path_to_tflite_model:
#     print(f"\nSearch completed! Model saved at: {path_to_tflite_model}")
# else:
#     print("\nNo feasible architecture found.")


import threading
import time
from IPython.display import clear_output

def clear_output_every(interval=10):
    while not stop_clear.is_set():
        time.sleep(interval)
        clear_output(wait=True)

stop_clear = threading.Event()
clear_thread = threading.Thread(target=clear_output_every, args=(10,))
clear_thread.start()

# ---- Run NAS ----
print("Starting search...")
path_to_tflite_model = colabNAS.search()

# ---- Stop clearing ----
stop_clear.set()
clear_thread.join()

# ---- Final message ----
clear_output(wait=True)
if path_to_tflite_model:
    print(f"Search completed! Model saved at: {path_to_tflite_model}")
else:
    print("No feasible architecture found.")


80/81 [============================>.] - ETA: 0s - loss: 1.3429 - accuracy: 0.3949
Epoch 20: val_accuracy did not improve from 0.40599
81/81 [==============================] - 0s 6ms/step - loss: 1.3422 - accuracy: 0.3943 - val_loss: 1.3204 - val_accuracy: 0.4024
Epoch 21/99
73/81 [==========================>...] - ETA: 0s - loss: 1.3421 - accuracy: 0.3947
Epoch 21: val_accuracy did not improve from 0.40599
81/81 [==============================] - 0s 5ms/step - loss: 1.3408 - accuracy: 0.3970 - val_loss: 1.3720 - val_accuracy: 0.3833
Epoch 22/99
70/81 [========================>.....] - ETA: 0s - loss: 1.3437 - accuracy: 0.3929
Epoch 22: val_accuracy did not improve from 0.40599
81/81 [==============================] - 0s 5ms/step - loss: 1.3398 - accuracy: 0.3951 - val_loss: 1.3143 - val_accuracy: 0.4060
Epoch 23/99
77/81 [===========================>..] - ETA: 0s - loss: 1.3361 - accuracy: 0.3916
Epoch 23: val_accuracy did not improve from 0.40599
81/81 [==============================

INFO:tensorflow:Assets written to: C:\Users\CHAKKA~1\AppData\Local\Temp\tmp77wm34gj\assets


INFO:tensorflow:Assets written to: C:\Users\CHAKKA~1\AppData\Local\Temp\tmp77wm34gj\assets
c:\Users\Chakkaphan\anaconda3\envs\tinyml\lib\site-packages\tensorflow\lite\python\convert.py:766: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


Error evaluating model k=2, c=0: 'PrefetchDataset' object has no attribute 'rebatch'



{'k': 2, 'c': 0, 'max_val_acc': -3}




k_1_c_0

81/81 [==============================] - 1s 8ms/step - loss: 1.8257 - accuracy: 0.2005 - val_loss: 1.6183 - val_accuracy: 0.1980


KeyboardInterrupt: 

In [ ]:
# Test the model if found
if path_to_tflite_model and Path(path_to_tflite_model).exists():
    # Create test dataset
    test_ds = tf.keras.utils.image_dataset_from_directory(
        directory=str(data_dir),
        labels='inferred',
        label_mode='categorical',
        color_mode='rgb',
        batch_size=32,
        image_size=input_shape[0:2],
        shuffle=True,
        seed=42,
        validation_split=0.8,
        subset='validation'
    )
    
    print("Testing model...")
    accuracy = test_tflite_model(path_to_tflite_model, test_ds)
    
    model_size = Path(path_to_tflite_model).stat().st_size
    print(f"Model size: {model_size/1024:.2f} KB")
    print(f"Model accuracy: {accuracy:.4f}")
else:
    print("No model to test.")

Found 3670 files belonging to 5 classes.
Using 2936 files for validation.
Testing model...


c:\Users\Chakkaphan\anaconda3\envs\tinyml\lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)



TFLite model test accuracy: 0.6080
Model size: 7.48 KB
Model accuracy: 0.6080
